In [27]:
from os.path import join
from data import build_corpus, build_corpus_origin
from utils import load_model, extend_maps, prepocess_data_for_lstmcrf
from models.util import sort_by_lengths, tensorized

In [28]:
bilstm_model = load_model("ckpts/bilstm.pkl")

In [29]:
bilstm_model

In [30]:
train_word_lists, train_tag_lists, word2id, tag2id = build_corpus_origin("train")
dev_word_lists, dev_tag_lists = build_corpus_origin("dev", make_vocab=False)
test_word_lists, test_tag_lists = build_corpus_origin("test", make_vocab=False)
bilstm_word2id, bilstm_tag2id = extend_maps(word2id, tag2id, for_crf=False)
# crf_word2id, crf_tag2id = extend_maps(word2id, tag2id, for_crf=True)
# train_word_lists, train_tag_lists = prepocess_data_for_lstmcrf(train_word_lists, train_tag_lists)
# dev_word_lists, dev_tag_lists = prepocess_data_for_lstmcrf(dev_word_lists, dev_tag_lists)
# test_word_lists, test_tag_lists = prepocess_data_for_lstmcrf(test_word_lists, test_tag_lists, test=True)

In [31]:
tag2id
# "B-NAME M-NAME E-NAME S-NAME 姓名"
# "B-CONT M_CONT E_CONT 国籍"
# "B-RACE M-RACE E-RACE S-RACE 民族"
# "B-TITLE M-TITLE E-TITLE 头衔"
# "B-EDU M-EDU E-EDU 学历"
# "B-ORG M-ORG E-ORG S-ORG 组织"
# "B-PRO M-PRO E-PRO 专业"
# "B-LOC M-LOC E-LOC 地点"

{'B-NAME': 0,
 'E-NAME': 1,
 'O': 2,
 'B-CONT': 3,
 'M-CONT': 4,
 'E-CONT': 5,
 'B-RACE': 6,
 'E-RACE': 7,
 'B-TITLE': 8,
 'M-TITLE': 9,
 'E-TITLE': 10,
 'B-EDU': 11,
 'M-EDU': 12,
 'E-EDU': 13,
 'B-ORG': 14,
 'M-ORG': 15,
 'E-ORG': 16,
 'M-NAME': 17,
 'B-PRO': 18,
 'M-PRO': 19,
 'E-PRO': 20,
 'S-RACE': 21,
 'S-NAME': 22,
 'B-LOC': 23,
 'M-LOC': 24,
 'E-LOC': 25,
 'M-RACE': 26,
 'S-ORG': 27,
 '<unk>': 28,
 '<pad>': 29}

In [32]:
word_lists, tag_lists, indices = sort_by_lengths(test_word_lists, test_tag_lists)

In [14]:
len(train_word_lists)

3821

In [15]:
train_word_lists[0]

['高',
 '勇',
 '：',
 '男',
 '，',
 '中',
 '国',
 '国',
 '籍',
 '，',
 '无',
 '境',
 '外',
 '居',
 '留',
 '权',
 '，']

In [16]:
tensorized_sents, lengths = tensorized([train_word_lists[0]], bilstm_word2id)

In [17]:
# 有cuda就用上面这一句
# tensorized_sents = tensorized_sents.cuda()
# 将数据转移到 CPU
tensorized_sents = tensorized_sents.cpu()

In [18]:
tensorized_sents.shape

torch.Size([1, 17])

In [19]:
batch_tagids = bilstm_model.model.test(tensorized_sents, lengths, bilstm_tag2id)

In [35]:
# 将id转化为标注
crf = False

pred_tag_lists = []
id2tag = dict((id_, tag) for tag, id_ in bilstm_tag2id.items())
for i, ids in enumerate(batch_tagids):
    tag_list = []

    if crf:
        for j in range(lengths[i] - 1):  # crf解码过程中，end被舍弃
            tag_list.append(id2tag[ids[j].item()])
    else:
        for j in range(lengths[i]):
            tag_list.append(id2tag[ids[j].item()])
    
    pred_tag_lists.append(tag_list)

In [36]:
pred_tag_lists

[['B-NAME',
  'E-NAME',
  'O',
  'O',
  'O',
  'B-CONT',
  'M-CONT',
  'M-CONT',
  'E-CONT',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O']]

In [38]:
#下面是对哈利波特中的句子进行测试

In [33]:
def seperate_ch(sequence):
    return [ch for ch in sequence]

In [34]:
test = seperate_ch('哈利看着窗外没说话')
test

['哈', '利', '看', '着', '窗', '外', '没', '说', '话']

In [35]:
tensorized_sents, lengths = tensorized([test], bilstm_word2id)

In [36]:
tensorized_sents = tensorized_sents.cpu()

In [37]:
tensorized_sents.shape

torch.Size([1, 9])

In [38]:
batch_tagids = bilstm_model.model.test(tensorized_sents, lengths, bilstm_tag2id)

In [39]:
# 将id转化为标注
crf = False

pred_tag_lists = []
id2tag = dict((id_, tag) for tag, id_ in bilstm_tag2id.items())
for i, ids in enumerate(batch_tagids):
    tag_list = []

    if crf:
        for j in range(lengths[i] - 1):  # crf解码过程中，end被舍弃
            tag_list.append(id2tag[ids[j].item()])
    else:
        for j in range(lengths[i]):
            tag_list.append(id2tag[ids[j].item()])

    pred_tag_lists.append(tag_list)

In [40]:
pred_tag_lists

[['B-NAME', 'E-NAME', 'O', 'O', 'O', 'O', 'O', '<unk>', 'O']]

In [ ]:
# 对哈利波特书本进行实体识别

In [1]:
with open("Harry_Potter.txt", "r", encoding="utf-8") as f:
    total_lines = [line.strip() for line in f.readlines()]

total_lines = [line for line in total_lines if line != '']

In [2]:
# 分句
cutLineFlag = ["？", "！", "。", "!"]
sentenceList = []
for words in total_lines:
    oneSentence = ""
    for word in words:
        if word not in cutLineFlag:
            oneSentence = oneSentence + word
        else:
            oneSentence = oneSentence + word
            if oneSentence.__len__() > 4:
                sentenceList.append(oneSentence.strip())
            oneSentence = ""

In [5]:
resultList = []
resultList = [seperate_ch(sentence) for sentence in sentenceList]

In [23]:
tensorized_sents, lengths = tensorized(resultList, bilstm_word2id)

In [24]:
tensorized_sents = tensorized_sents.cpu()

In [25]:
batch_tagids = bilstm_model.model.test(tensorized_sents, lengths, bilstm_tag2id)

In [26]:
# 将id转化为标注
crf = False

pred_tag_lists = []
id2tag = dict((id_, tag) for tag, id_ in bilstm_tag2id.items())
for i, ids in enumerate(batch_tagids):
    tag_list = []

    if crf:
        for j in range(lengths[i] - 1):  # crf解码过程中，end被舍弃
            tag_list.append(id2tag[ids[j].item()])
    else:
        for j in range(lengths[i]):
            tag_list.append(id2tag[ids[j].item()])

    pred_tag_lists.append(tag_list)